In [1]:
#下面是用来识别的函数
import numpy as np
from PIL import Image as im
import matplotlib.pyplot as plt
import cv2
import rclpy
from rclpy.node import Node
from geometry_msgs.msg import Twist
import numpy as np
from std_msgs.msg import String
from sensor_msgs.msg import Joy
import time
from gazebo_msgs.srv import SpawnEntity
from ament_index_python.packages import get_package_share_directory
from sensor_msgs.msg import Image
from gazebo_msgs.srv import SetEntityState,GetEntityState
from tqdm import tqdm
rclpy.init()
if_callback = 0
#取图工具
class LeftSubscriber(Node):
    def __init__(self):
        super().__init__('minimal_subscriber')
        self.subscription = self.create_subscription(
            Image,
            '/demo_cam/mycamera/left/image_demo',
            self.listener_callback,
            1)
        self.subscription  # prevent unused variable warning
        #self.recognizer = Recognizer()

    def listener_callback(self,msg):
        global if_callback
        self.left_img = np.array(msg.data)
        self.left_img = self.left_img.reshape((640,960,3))
        if_callback +=1
        print("In callback")

leftsubscriber = LeftSubscriber()#实例化
class Recognizer_big(object):
    def __init__(self,world):
        if world == 'world_1':
            self.tile_area = [3.4e7,4e7]
        elif world == 'world_2':
            self.tile_area = [1.7e7,2e7]
    
    def angle_cos(self,p0, p1, p2):
        d1, d2 = (p0-p1).astype('float'), (p2-p1).astype('float')
        return abs( np.dot(d1, d2) / np.sqrt( np.dot(d1, d1)*np.dot(d2, d2) ) )
    
    def find_squares(self,contours):
        squares = []#make a void list,to collect the contours which match the condition
        for cnt in contours:
            cnt_len = cv2.arcLength(cnt, True) #compute the perimeter of the contour
            cnt = cv2.approxPolyDP(cnt, 0.02*cnt_len, True) #Polygon approximation
            #if the approximated polygon has 4 sides,and area>1000 and a convex polygon
            if len(cnt) == 4 and cv2.contourArea(cnt) > self.tile_area[0] and cv2.contourArea(cnt)<self.tile_area[1] and cv2.isContourConvex(cnt):
                M = cv2.moments(cnt) #compute the moment of the contour
                cx = int(M['m10']/M['m00'])
                cy = int(M['m01']/M['m00'])#compute the center
                cnt = cnt.reshape(-1, 2)
                max_cos = np.max([self.angle_cos( cnt[i], cnt[(i+1) % 4], cnt[(i+2) % 4] ) for i in range(4)])
                if max_cos < 0.1:# if nearly right angle
                    squares.append(cnt)
        return squares

    def compute_center(self,contour):
        M = cv2.moments(contour)  
        center_x = int(M["m10"] / M["m00"])
        center_y = int(M["m01"] / M["m00"])
        return center_x,center_y

    def imgcor2boxcor(self,x,y):#transformation from coordinate on the image to coordinate of the physic world 
        box_x = ((24090-y+6000)/10)/1000
        box_y = ((15060-x)/10)/1000
        return box_x,box_y
    def durch_translation(self,img):
        #perspective transform
        M = np.load('/home/jsy/expri_data/M_960.npy')#load the matrix for perspective transfomation
        img = cv2.warpPerspective(img,M,(30121,24091))#project the image to a big picture, each pixel is 1mm 
        gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)#Grayscale
        ret, binary = cv2.threshold(gray,205,255,cv2.THRESH_BINARY)#Binarization，for this task when the thresh set between 200-210,with the best effect
        contours, hierarchy = cv2.findContours(binary,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
        contours = self.find_squares(contours)
        #compute the angle of the conotours with the horizon
        angles = []
        centers = []
        w_h = []
        for i in contours:
            rect = cv2.minAreaRect(i)#fit a rectangle with the minimal area,in ideal,this rectangle should be the contour self
            angles.append(np.minimum(np.absolute(rect[2]),np.absolute(90+rect[2])))
            #sins.append(np.absolute(np.sin(np.deg2rad(rect[2]))))#之所以用sin而不是角度是因为正对着时会出现0和-90两种数字
        angle = np.mean(angles)#
        cos = np.cos(np.deg2rad(angle))
        sin = np.sin(np.deg2rad(angle))
        #compute how much it should translate,603 is the sollwert(603mm is the distance zwischen tiles)
        tx = 6030*cos#compute translation distance along X-axis
        ty = 6030*sin#compute translation distance along Y-axis
        #compute new contours after translation, if beyond the image, throw it
        new_contours = [] 
        for i in range(len(contours)):
            a_contour = contours[i]+np.array([tx,ty])
            if a_contour[a_contour[:,0]>30120].size>0 or a_contour[a_contour[:,1]>24090].size>0:
                pass
            else:
                new_contours.append(a_contour)

        new_contours = np.int32(new_contours)

        #compare all new contour with feature of ground, if the color of a contours'center is gray, that means there is no tile
        next_contours = []
        img_coordinate = []
        for i in new_contours:
            x,y = self.compute_center(i)
            #if color on center is gray, then save it in a list
            if np.std(img[y][x])<0.1:#attention! Numpy index of a image is image[y][x],x is horizon,y is vertical
                next_contours.append(i)
                img_coordinate.append((x,y))
        next_contours = np.int32(next_contours)
        #frome image coordinate to coordinate relative to the center of the Fliesenleger
        box_coordinate = []
        for i in img_coordinate:
            box_x,box_y = self.imgcor2boxcor(i[0],i[1])
            box_coordinate.append([box_x,box_y,-0.845,0.,0.,angle])

        for i in img_coordinate: 
            img = cv2.circle(img,i, 5, (255, 0, 0), -1)#draw the centers of the choosen contour
        cv2.drawContours(img,next_contours,-1,(0,0,255),3)#draw the choosen contour 
        self.img_contours = im.fromarray(img)
        
        return box_coordinate
    
class Set_State_Client(Node):
    def __init__(self):
        super().__init__('set_state_client')
        self.cli = self.create_client(SetEntityState,'/set_entity_state')
        #while not self.cli.wait_for_service(timeout_sec=1.0):
        #    self.get_logger().info('service not available, waiting again...')
        self.req = SetEntityState.Request()

    def send_request(self):
        self.req.state.name = 'nested_model'
        self.future = self.cli.call_async(self.req)
        
class Get_State_Client(Node):
    def __init__(self):
        super().__init__('get_state_client')
        self.cli = self.create_client(GetEntityState,'/get_entity_state')
        #while not self.cli.wait_for_service(timeout_sec=1.0):
        #    self.get_logger().info('service not available, waiting again...')
        self.req = GetEntityState.Request()
        self.req.name = 'nested_model'
        self.req.reference_frame = 'world'

    def send_request(self):
        self.future = self.cli.call_async(self.req)
    

In [2]:
#不过有些位置识别不了，所以会有很多为空
#而且这里有个问题，就是照片刷新率跟不上执行速度，也就是说调整姿势后，用subscriber得到的照片有时候会是上一时刻的
start = time.time()
result_list = []
points = np.load('/home/jsy/expri_data/可用的点.npy')
real_x = 1.206
real_y = 0.603
counter_1 = 0
counter_2 = 0
set_state_client = Set_State_Client()
get_state_client = Get_State_Client()
recognizer = Recognizer_big('world_1')
for pos in tqdm(points):
    x = pos[0]
    y = pos[1]
    z = pos[2]
    #rclpy.spin_once(leftsubscriber)
    set_state_client.req.state.pose.position.x = x#给request的x赋值
    set_state_client.req.state.pose.position.y = y#给request的y赋值
    set_state_client.req.state.pose.orientation.z = np.sin(np.deg2rad(-180.+z)/2)#给出绕z轴角度转换成四元数的z为sin(alpha/2)
    set_state_client.req.state.pose.orientation.w = np.cos(np.deg2rad(-180.+z)/2)#给出绕z轴角度转换成四元数的w为cos(alpha/2)
    set_state_client.send_request()#发送调整位姿请求
    rclpy.spin_until_future_complete(set_state_client,set_state_client.future)#必须用until future complete才不会出错
    print(set_state_client.future.result().success)
    get_state_client.send_request()
    rclpy.spin_until_future_complete(get_state_client,get_state_client.future)#获取位资
    print(get_state_client.future.result().success)
    while if_callback<3:
        rclpy.spin_once(leftsubscriber)#每运行一次就取回一个图在leftsubscriber.left_img里
    if_callback=0
    box_coordinate = recognizer.durch_translation(leftsubscriber.left_img)
    counter_1+=1
    if box_coordinate!= []:
        angle_rad = np.deg2rad(-180.+z)
        expri_x = np.cos(angle_rad)*box_coordinate[0][0]-np.sin(angle_rad)*box_coordinate[0][1]+x
        expri_y = np.sin(angle_rad)*box_coordinate[0][0]+np.cos(angle_rad)*box_coordinate[0][1]+y
        error_x = expri_x-real_x#计算x方向的误差
        error_y = expri_y-real_y#计算y方向的误差
        error_center = np.sqrt(np.square(error_x)+np.square(error_y))#计算中心点距离的绝对误差
        error_angle = box_coordinate[0][5]-z
        cam_x = np.cos(angle_rad)*0.45-np.sin(angle_rad)*0.0315+x
        cam_y = np.sin(angle_rad)*0.45+np.cos(angle_rad)*0.0315+y
        result =[x,expri_x,y,expri_y,z,box_coordinate[0][5],error_x,error_y,error_center,error_angle,cam_x,cam_y]#返回每次实验的机器位置，旋转角度，以及测定的误差和摄像头的位置
        result_list.append(result)
        counter_2+=1
        print(result)
end = time.time()
print('测试完毕，共测试了 %d 次，其中识别出砖的共 %d 次' % (counter_1,counter_2))
print('共耗时 '+str(end-start)+'秒')

  0%|          | 0/7666 [00:00<?, ?it/s]

True
True
In callback
In callback
In callback


  0%|          | 1/7666 [00:02<5:40:45,  2.67s/it]

[2.0, 1.2070581243950675, 1.2500000000000004, 0.6020156159967034, 30.0, 30.13252894083659, 0.0010581243950675834, -0.00098438400329659, 0.0014452124762066511, 0.13252894083658973, 1.6260385682970027, 0.9977201997807906]
True
True
In callback
In callback
In callback


  0%|          | 2/7666 [00:05<5:40:53,  2.67s/it]

...

True
True
In callback
In callback
In callback


100%|█████████▉| 7665/7666 [3:42:40<00:01,  1.73s/it]

True
True
In callback
In callback
In callback


100%|██████████| 7666/7666 [3:42:42<00:00,  1.74s/it]

测试完毕，共测试了 7666 次，其中识别出砖的共 502 次
共耗时 13362.425887823105秒


In [3]:
data_1k = np.vstack(result_list)

In [4]:
data_1k.shape

(502, 12)

In [5]:
np.save('/home/jsy/expri_data/data_1k.npy',data_1k)